### Adding more factors on NUTS2 level

- The notebook begins with importing data from multiple sources, including Excel and CSV files. It involves loading datasets and merging new factors on NUTS 2 for Poland. The focus on different socio-economic indicators will help in conducting a comprehensive analysis of regions in Poland. We will use the final dataset to the modeling part.
- The notebook reflects a focus on adding new factors to the existing modeling framework for Poland, particularly at the NUTS 3 level. Key considerations include the temporal availability of these factors, with a notable limitation that the new factors are available only for 2014-2020, unlike the existing dataset covering 2007-2020.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load datasets
df = pd.read_excel("full_data_pl.xlsx")
wynagrodzenie = pd.read_csv('SZKO_4058_CTAB_20231123195156.csv', delimiter = ';')
mapping_data = pd.read_excel("clean_mapping_pl.xlsx")
wynagrodzenie = wynagrodzenie[['Kod', 'Nazwa']]
wynagrodzenie

,Kod,Nazwa
0,11212000000,PODREGION KRAKOWSKI
1,11212100000,PODREGION MIASTO KRAKÓW
2,11212200000,PODREGION NOWOSĄDECKI
3,11212300000,PODREGION OŚWIĘCIMSKI
4,11212400000,PODREGION TARNOWSKI
...,...,...
68,71422600000,PODREGION OSTROŁĘCKI
69,71422700000,PODREGION RADOMSKI
70,71427000000,PODREGION PŁOCKI
71,71427100000,PODREGION SIEDLECKI


In [3]:
# There are some misspellings that might cause and issue in merging the datasets. We will correct them
replacement_dict = {
    'PODREGION OLSZYŃSKI': 'PODREGION OLSZTYŃSKI',
    'PODREGION SANDOMIERSKO - JĘDRZEJOWSKI': 'PODREGION SANDOMIERSKO-JĘDRZEJOWSKI',
    'PODREGION WARSZAWA WSCHODNIA':'PODREGION WARSZAWSKI WSCHODNI',
    'PODREGION ŁOMŻYNSKI':'PODREGION ŁOMŻYŃSKI',
    'PODREGION KOSZALINSKI':'PODREGION KOSZALIŃSKI',
    'PODREGION KROSNIEŃSKI':'PODREGION KROŚNIEŃSKI',
    'PODREGION GLIWCKI':'PODREGION GLIWICKI',
    'PODREGION GRUDZIADZKI':'PODREGION GRUDZIĄDZKI'

}

# Replace values in the 'NUTS3' column
mapping_data['NUTS3'] = mapping_data['NUTS3'].replace(replacement_dict)
mapping_data

,Unnamed: 0,NUTS1,NUTS1 CODE,NUTS2,NUTS2 CODE,NUTS3,NUTS3 CODE
0,0,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION WROCŁAWSKI,PL619
1,1,MAKROREGION WSCHODNI,PL8,REGION PODLASKIE,PL84,PODREGION SUWALSKI,PL843
2,2,MAKROREGION PÓŁNOCNY,PL6,REGION WARMIŃSKO-MAZURSKIE,PL62,PODREGION OLSZTYŃSKI,PL622
3,3,MAKROREGION POŁUDNIOWY,PL2,REGION ŚLĄSKIE,PL22,PODREGION SOSNOWIECKI,PL22B
4,4,MAKROREGION CENTRALNY,PL7,REGION ŁÓDZKIE,PL71,PODREGION PIOTRKOWSKI,PL713
...,...,...,...,...,...,...,...
67,67,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION WIELKOPOLSKIE,PL41,PODREGION MIASTO POZNAŃ,PL415
68,68,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION ŚWIECKI,PL618
69,69,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION ZACHODNIOPOMORSKIE,PL42,PODREGION MIASTO SZCZECIN,PL424
70,70,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION WARSZAWSKI STOŁECZNY,PL91,PODREGION MIASTO WARSZAWA,PL911


In [4]:
# Make a copy of the DataFrame
wynagrodzenie_long = wynagrodzenie.copy()

In [5]:
# Check unique values
wynagrodzenie_long['Nazwa'].unique()

array(['PODREGION KRAKOWSKI', 'PODREGION MIASTO KRAKÓW',
       'PODREGION NOWOSĄDECKI', 'PODREGION OŚWIĘCIMSKI',
       'PODREGION TARNOWSKI', 'PODREGION NOWOTARSKI', 'PODREGION BIELSKI',
       'PODREGION BYTOMSKI', 'PODREGION CZĘSTOCHOWSKI',
       'PODREGION GLIWICKI', 'PODREGION KATOWICKI', 'PODREGION RYBNICKI',
       'PODREGION SOSNOWIECKI', 'PODREGION TYSKI', 'PODREGION GORZOWSKI',
       'PODREGION ZIELONOGÓRSKI', 'PODREGION KALISKI',
       'PODREGION KONIŃSKI', 'PODREGION LESZCZYŃSKI', 'PODREGION PILSKI',
       'PODREGION POZNAŃSKI', 'PODREGION MIASTO POZNAŃ',
       'PODREGION KOSZALIŃSKI', 'PODREGION SZCZECINECKO-PYRZYCKI',
       'PODREGION MIASTO SZCZECIN', 'PODREGION SZCZECIŃSKI',
       'PODREGION JELENIOGÓRSKI', 'PODREGION LEGNICKO-GŁOGOWSKI',
       'PODREGION WAŁBRZYSKI', 'PODREGION WROCŁAWSKI',
       'PODREGION MIASTO WROCŁAW', 'PODREGION NYSKI', 'PODREGION OPOLSKI',
       'PODREGION BYDGOSKO-TORUŃSKI', 'PODREGION GRUDZIĄDZKI',
       'PODREGION WŁOCŁAWSKI', 'PO

In [6]:
# Check unique values
unique_in_mapping = set(wynagrodzenie_long['Nazwa']) - set(mapping_data['NUTS3'])
unique_in_mapping

{'PODREGION WARSZAWSKI ZACHODNI', 'PODREGION WŁOCŁAWSKI'}

In [7]:
# Check unique values
unique_in_mapping = set(mapping_data['NUTS3']) - set(wynagrodzenie_long['Nazwa'])
unique_in_mapping

set()

In [8]:
# Check unique values
mapping_data['NUTS3']

0          PODREGION WROCŁAWSKI
1            PODREGION SUWALSKI
2          PODREGION OLSZTYŃSKI
3         PODREGION SOSNOWIECKI
4         PODREGION PIOTRKOWSKI
                ...            
67      PODREGION MIASTO POZNAŃ
68            PODREGION ŚWIECKI
69    PODREGION MIASTO SZCZECIN
70    PODREGION MIASTO WARSZAWA
71     PODREGION MIASTO WROCŁAW
Name: NUTS3, Length: 72, dtype: object

In [9]:
wynagrodzenie_long['Nazwa']

0         PODREGION KRAKOWSKI
1     PODREGION MIASTO KRAKÓW
2       PODREGION NOWOSĄDECKI
3       PODREGION OŚWIĘCIMSKI
4         PODREGION TARNOWSKI
               ...           
68       PODREGION OSTROŁĘCKI
69         PODREGION RADOMSKI
70           PODREGION PŁOCKI
71        PODREGION SIEDLECKI
72      PODREGION ŻYRARDOWSKI
Name: Nazwa, Length: 73, dtype: object

In [10]:
common_elements = set(mapping_data['NUTS3']).intersection(set(wynagrodzenie_long['Nazwa']))
len(common_elements)

71

In [11]:
# Merge with mapping_data
merged_df = pd.merge(mapping_data, wynagrodzenie_long, left_on=['NUTS3'], right_on=['Nazwa'], how='left')

merged_df=merged_df.drop(["Nazwa"], axis = 1)

merged_df

,Unnamed: 0,NUTS1,NUTS1 CODE,NUTS2,NUTS2 CODE,NUTS3,NUTS3 CODE,Kod
0,0,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION WROCŁAWSKI,PL619,30210400000
1,1,MAKROREGION WSCHODNI,PL8,REGION PODLASKIE,PL84,PODREGION SUWALSKI,PL843,62013900000
2,2,MAKROREGION PÓŁNOCNY,PL6,REGION WARMIŃSKO-MAZURSKIE,PL62,PODREGION OLSZTYŃSKI,PL622,42815600000
3,3,MAKROREGION POŁUDNIOWY,PL2,REGION ŚLĄSKIE,PL22,PODREGION SOSNOWIECKI,PL22B,12415000000
4,4,MAKROREGION CENTRALNY,PL7,REGION ŁÓDZKIE,PL71,PODREGION PIOTRKOWSKI,PL713,51011700000
...,...,...,...,...,...,...,...,...
67,67,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION WIELKOPOLSKIE,PL41,PODREGION MIASTO POZNAŃ,PL415,23016200000
68,68,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION ŚWIECKI,PL618,40416800000
69,69,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION ZACHODNIOPOMORSKIE,PL42,PODREGION MIASTO SZCZECIN,PL424,23216500000
70,70,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION WARSZAWSKI STOŁECZNY,PL91,PODREGION MIASTO WARSZAWA,PL911,71412800000


In [29]:
# We are missing one code and region that was not in mapping, we will add it manually now
new_row = {'NUTS1': 'MAKROREGION WOJEWÓDZTWO MAZOWIECKIE', 'NUTS1 CODE': 'PL9', 'NUTS2':'REGION WARSZAWSKI STOŁECZNY','NUTS2 CODE' : 'PL91', 'NUTS3':'PODREGION WARSZAWSKI ZACHODNI', 'NUTS3 CODE':'PL913', 'Kod': '71413000000'}

# Append the new row to the DataFrame
merged_df = merged_df.append(new_row, ignore_index=True)

merged_df

/var/folders/vn/7p3t4_3n02gcv591cgws1gw80000gn/T/ipykernel_18152/2282704114.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(new_row, ignore_index=True)


,Unnamed: 0,NUTS1,NUTS1 CODE,NUTS2,NUTS2 CODE,NUTS3,NUTS3 CODE,Kod
0,0.0,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION WROCŁAWSKI,PL619,30210400000
1,1.0,MAKROREGION WSCHODNI,PL8,REGION PODLASKIE,PL84,PODREGION SUWALSKI,PL843,62013900000
2,2.0,MAKROREGION PÓŁNOCNY,PL6,REGION WARMIŃSKO-MAZURSKIE,PL62,PODREGION OLSZTYŃSKI,PL622,42815600000
3,3.0,MAKROREGION POŁUDNIOWY,PL2,REGION ŚLĄSKIE,PL22,PODREGION SOSNOWIECKI,PL22B,12415000000
4,4.0,MAKROREGION CENTRALNY,PL7,REGION ŁÓDZKIE,PL71,PODREGION PIOTRKOWSKI,PL713,51011700000
...,...,...,...,...,...,...,...,...
69,69.0,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION ZACHODNIOPOMORSKIE,PL42,PODREGION MIASTO SZCZECIN,PL424,23216500000
70,70.0,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION WARSZAWSKI STOŁECZNY,PL91,PODREGION MIASTO WARSZAWA,PL911,71412800000
71,71.0,MAKROREGION POŁUDNIOWO-ZACHODNI,PL5,REGION DOLNOŚLĄSKIE,PL51,PODREGION MIASTO WROCŁAW,PL514,30210500000
72,NaN,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION WARSZAWSKI STOŁECZNY,PL91,PODREGION WARSZAWSKI ZACHODNI,PL913,71413000000


### We will add Kod column to our full Poland data (df) to have this system there too

In [30]:
df

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC)
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916
1018,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326
1019,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665
1020,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483


In [31]:
common_elements = set(df['NUTS3 CODE']).intersection(set(merged_df['NUTS3 CODE']))
len(common_elements)

73

In [32]:
# Create a new DataFrame with only 'NUTS3 CODE' and 'Kod' columns from merged_df
merged_df_subset = merged_df[['NUTS3 CODE', 'Kod']]

# Merge df with merged_df_subset on 'NUTS3 CODE'
final = df.merge(merged_df_subset, on='NUTS3 CODE', how='left')

final

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000


In [33]:
# Check missing values
missing = final[final['Kod'].isnull()]
missing

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod


#### Adding new factor - przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej 2007 - 2020

In [34]:
wyna = pd.read_csv('WYNA_2497_CTAB_20231126095948.csv', delimiter=';')
wyna

,Kod,Nazwa,przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2007;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2008;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2009;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2010;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2011;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2012;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2013;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2014;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2015;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2016;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2017;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2018;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2019;[%],przeciętne miesięczne wynagrodzenia brutto w relacji do średniej krajowej (Polska=100);2020;[%],Unnamed: 16
0,201000,Powiat bolesławiecki,"81,1","78,4","79,3","81,2","81,0","81,6","83,3","85,5","85,5","85,4","86,2","87,2","87,7","88,5",NaN
1,202000,Powiat dzierżoniowski,"78,4","78,8","79,8","82,4","83,4","83,1","81,4","82,4","82,1","83,0","84,5","83,3","86,5","84,9",NaN
2,203000,Powiat głogowski,"85,6","83,8","86,3","84,8","83,5","83,4","85,6","86,4","87,1","86,8","86,5","84,3","85,5","86,2",NaN
3,204000,Powiat górowski,"73,3","75,2","74,3","75,6","76,0","77,7","79,1","78,5","78,1","78,9","77,0","78,9","76,8","80,2",NaN
4,205000,Powiat jaworski,"76,5","75,0","75,6","77,3","78,3","79,7","78,4","78,5","80,1","82,7","83,0","83,3","85,1","89,3",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,3217000,Powiat wałecki,"83,2","80,4","80,8","82,3","82,2","82,6","83,3","83,3","83,0","82,9","83,1","83,7","84,2","84,7",NaN
376,3218000,Powiat łobeski,"74,1","74,7","75,4","76,9","78,4","82,3","80,5","82,2","82,4","83,8","80,9","84,5","83,4","85,3",NaN
377,3261000,Powiat m. Koszalin,"88,4","87,8","88,4","89,4","87,8","88,8","90,1","89,1","87,8","89,0","88,3","86,5","89,3","90,0",NaN
378,3262000,Powiat m. Szczecin,"103,8","104,2","104,7","104,4","103,8","103,7","103,9","104,3","105,6","105,8","104,5","103,6","104,4","103,1",NaN


In [35]:
# Reverse columns 
id_vars = ['Kod', 'Nazwa']

value_vars = [col for col in wyna.columns if col.startswith('przeciętne miesięczne wynagrodzenia')]

# Unpivot the DataFrame
wyna = wyna.melt(id_vars=id_vars, value_vars=value_vars, var_name='TIME_PERIOD', value_name='Wynagrodzenie %')

# Extract the year from the 'TIME_PERIOD' column
wyna['TIME_PERIOD'] = wyna['TIME_PERIOD'].str.extract('(\d{4})')
wyna

,Kod,Nazwa,TIME_PERIOD,Wynagrodzenie %
0,201000,Powiat bolesławiecki,2007,"81,1"
1,202000,Powiat dzierżoniowski,2007,"78,4"
2,203000,Powiat głogowski,2007,"85,6"
3,204000,Powiat górowski,2007,"73,3"
4,205000,Powiat jaworski,2007,"76,5"
...,...,...,...,...
5315,3217000,Powiat wałecki,2020,"84,7"
5316,3218000,Powiat łobeski,2020,"85,3"
5317,3261000,Powiat m. Koszalin,2020,"90,0"
5318,3262000,Powiat m. Szczecin,2020,"103,1"


In [36]:
len(wyna['Kod'].unique())

380

In [37]:
len(final['Kod'].unique())

72

In [38]:
common_elements = set(wyna['Kod']).intersection(set(final['Kod']))
len(common_elements)

0

In [39]:
# Create a new DataFrame with only 'Kod' and 'TIME_PERIOD' columns from merged_df
subset = wyna[['Kod', 'TIME_PERIOD', 'Wynagrodzenie %']].copy()

# Convert 'Kod' and 'TIME_PERIOD' to strings to ensure matching types for merge
subset['Kod'] = subset['Kod'].astype(str)
final['Kod'] = final['Kod'].astype(str)
subset['TIME_PERIOD'] = subset['TIME_PERIOD'].astype(str)
final['TIME_PERIOD'] = final['TIME_PERIOD'].astype(str)

# Merge df with subset on 'Kod' and 'TIME_PERIOD'
one_factor = final.merge(subset, on=['Kod', 'TIME_PERIOD'], how='left')
one_factor

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Wynagrodzenie %
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000,NaN
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000,NaN
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000,NaN
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000,NaN
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000,NaN
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000,NaN
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000,NaN


In [40]:
# Check missing values
missing = one_factor[one_factor['Wynagrodzenie %'].isnull()]
missing

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Wynagrodzenie %
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000,NaN
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000,NaN
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000,NaN
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000,NaN
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000,NaN
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000,NaN
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000,NaN


In [42]:
# 
df['NUTS3 CODE'] = df['NUTS3 CODE'].astype(str).str.strip()
merged_df['NUTS3 CODE'] = merged_df['NUTS3 CODE'].astype(str).str.strip()

# Merging df with selected columns from merged_df
merged_result = df.merge(
    merged_df[['Kod', 'NUTS3 CODE']],
    on='NUTS3 CODE',
    how='left'
)

merged_df

,Unnamed: 0,NUTS1,NUTS1 CODE,NUTS2,NUTS2 CODE,NUTS3,NUTS3 CODE,Kod
0,0.0,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION WROCŁAWSKI,PL619,30210400000
1,1.0,MAKROREGION WSCHODNI,PL8,REGION PODLASKIE,PL84,PODREGION SUWALSKI,PL843,62013900000
2,2.0,MAKROREGION PÓŁNOCNY,PL6,REGION WARMIŃSKO-MAZURSKIE,PL62,PODREGION OLSZTYŃSKI,PL622,42815600000
3,3.0,MAKROREGION POŁUDNIOWY,PL2,REGION ŚLĄSKIE,PL22,PODREGION SOSNOWIECKI,PL22B,12415000000
4,4.0,MAKROREGION CENTRALNY,PL7,REGION ŁÓDZKIE,PL71,PODREGION PIOTRKOWSKI,PL713,51011700000
...,...,...,...,...,...,...,...,...
69,69.0,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION ZACHODNIOPOMORSKIE,PL42,PODREGION MIASTO SZCZECIN,PL424,23216500000
70,70.0,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION WARSZAWSKI STOŁECZNY,PL91,PODREGION MIASTO WARSZAWA,PL911,71412800000
71,71.0,MAKROREGION POŁUDNIOWO-ZACHODNI,PL5,REGION DOLNOŚLĄSKIE,PL51,PODREGION MIASTO WROCŁAW,PL514,30210500000
72,NaN,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION WARSZAWSKI STOŁECZNY,PL91,PODREGION WARSZAWSKI ZACHODNI,PL913,71413000000


In [43]:
mis =merged_result[merged_result['Kod'].isnull()]
mis

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod


In [45]:
# Count the total number of nulls in each column of 'one_factor'
null_counts = one_factor.isna().sum()

# Display the count of nulls for each column
print(null_counts)

Unnamed: 0              0
NUTS3 CODE              0
TIME_PERIOD             0
POP (THS)               0
EMP (THS)               0
EUR_HAB                 0
EUR_HAB_EU27_2020       0
MIO_EUR                 0
MIO_NAC                 0
MIO_PPS_EU27_2020       0
PPS_EU27_2020_HAB       0
PPS_HAB_EU27_2020       0
GVA (MIO_EUR)           0
GVA (MIO_NAC)           0
Kod                     0
Wynagrodzenie %      1036
dtype: int64


#### Factor 1 - Relacja liczby studentów do ludności w wieku 19-24 lata, 2008 - 2020

In [47]:
pp = pd.read_csv('SZKO_4058_CTAB_20231123195156.csv', delimiter=';')
pp

,Kod,Nazwa,relacja liczby studentów do ludności w wieku 19-24 lata;2008;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2009;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2010;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2011;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2012;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2013;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2014;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2015;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2016;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2017;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2018;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2019;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2020;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2021;[-],relacja liczby studentów do ludności w wieku 19-24 lata;2022;[-],Unnamed: 17
0,11212000000,PODREGION KRAKOWSKI,"1,5","2,2","2,5","2,5","2,2","2,4","2,4","2,1","2,0","0,2","2,1","2,3","2,5","2,4","2,5",NaN
1,11212100000,PODREGION MIASTO KRAKÓW,"255,5","272,2","254,7","272,8","296,1","301,5","320,6","351,0","364,6","352,1","351,7","342,6","253,8","282,9","311,7",NaN
2,11212200000,PODREGION NOWOSĄDECKI,"17,1","17,4","18,0","16,5","14,7","13,0","11,5","10,6","9,9","10,2","10,3","10,9","13,7","16,6","22,1",NaN
3,11212300000,PODREGION OŚWIĘCIMSKI,"3,8","4,5","5,0","4,4","4,7","4,1","4,2","3,9","4,0","4,7","5,5","5,7","6,7","6,4","6,1",NaN
4,11212400000,PODREGION TARNOWSKI,"22,4","23,2","21,8","18,8","16,7","15,8","14,9","13,7","13,4","14,4","14,6","15,0","16,3","17,0","16,0",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,71422600000,PODREGION OSTROŁĘCKI,"8,8","7,3","7,1","5,7","4,4","3,3","2,9","2,4","2,1","2,0","1,8","1,9","2,4","2,7","2,8",NaN
69,71422700000,PODREGION RADOMSKI,"25,9","26,1","27,2","25,0","24,1","21,9","20,3","19,4","18,2","16,4","15,8","17,0","18,6","20,4","21,7",NaN
70,71427000000,PODREGION PŁOCKI,"33,6","32,1","31,1","29,7","27,8","25,9","25,2","23,9","23,4","23,5","22,9","22,4","23,2","22,0","20,8",NaN
71,71427100000,PODREGION SIEDLECKI,"31,5","32,2","34,4","30,9","27,7","25,0","23,4","20,6","19,1","19,6","19,9","20,7","23,3","23,7","23,3",NaN


In [48]:
id_vars = ['Kod', 'Nazwa']

# Specify the value_vars, which are the columns to melt
# These columns start with 'relacja liczby studentów do ludności w wieku 19-24 lata'
value_vars = [col for col in pp.columns if col.startswith('relacja liczby studentów do ludności w wieku 19-24 lata')]

# Performing the melt operation
pp = pp.melt(id_vars=id_vars, value_vars=value_vars, var_name='TIME_PERIOD', value_name='Studenci do Ludności w wieku 19-24')

# Extracting the year from the 'TIME_PERIOD' column
# The year is between two semicolons (;), so we split and get the second element ([1])
pp['TIME_PERIOD'] = pp['TIME_PERIOD'].apply(lambda x: x.split(';')[1])

# Verify the result
pp

,Kod,Nazwa,TIME_PERIOD,Studenci do Ludności w wieku 19-24
0,11212000000,PODREGION KRAKOWSKI,2008,"1,5"
1,11212100000,PODREGION MIASTO KRAKÓW,2008,"255,5"
2,11212200000,PODREGION NOWOSĄDECKI,2008,"17,1"
3,11212300000,PODREGION OŚWIĘCIMSKI,2008,"3,8"
4,11212400000,PODREGION TARNOWSKI,2008,"22,4"
...,...,...,...,...
1090,71422600000,PODREGION OSTROŁĘCKI,2022,"2,8"
1091,71422700000,PODREGION RADOMSKI,2022,"21,7"
1092,71427000000,PODREGION PŁOCKI,2022,"20,8"
1093,71427100000,PODREGION SIEDLECKI,2022,"23,3"


In [49]:
pp['Studenci do Ludności w wieku 19-24'] = pp['Studenci do Ludności w wieku 19-24'].str.replace(',', '.')
pp['Studenci do Ludności w wieku 19-24'] = pp['Studenci do Ludności w wieku 19-24'].astype(float)
nan_rows = pp[pp['Studenci do Ludności w wieku 19-24'].isna()]
nan_rows

,Kod,Nazwa,TIME_PERIOD,Studenci do Ludności w wieku 19-24


In [50]:
# Create a new DataFrame with only 'Kod' and 'TIME_PERIOD' columns from merged_df
subset = pp[['Kod', 'TIME_PERIOD', 'Studenci do Ludności w wieku 19-24']].copy()

# Convert 'Kod' and 'TIME_PERIOD' to strings to ensure matching types for merge
subset['Kod'] = subset['Kod'].astype(str)
final['Kod'] = final['Kod'].astype(str)
subset['TIME_PERIOD'] = subset['TIME_PERIOD'].astype(str)
final['TIME_PERIOD'] = final['TIME_PERIOD'].astype(str)

# Merge df with subset on 'Kod' and 'TIME_PERIOD'
one_factor = final.merge(subset, on=['Kod', 'TIME_PERIOD'], how='left')
one_factor

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Studenci do Ludności w wieku 19-24
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000,255.5
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000,272.2
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000,254.7
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000,272.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000,3.8
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000,5.5
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000,6.7
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000,2.1


In [51]:
one_factor

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Studenci do Ludności w wieku 19-24
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000,255.5
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000,272.2
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000,254.7
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000,272.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000,3.8
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000,5.5
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000,6.7
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000,2.1


In [52]:
# Count the total number of nulls in each column of 'one_factor'
null_counts = one_factor.isnull().sum()

null_counts

Unnamed: 0                             0
NUTS3 CODE                             0
TIME_PERIOD                            0
POP (THS)                              0
EMP (THS)                              0
EUR_HAB                                0
EUR_HAB_EU27_2020                      0
MIO_EUR                                0
MIO_NAC                                0
MIO_PPS_EU27_2020                      0
PPS_EU27_2020_HAB                      0
PPS_HAB_EU27_2020                      0
GVA (MIO_EUR)                          0
GVA (MIO_NAC)                          0
Kod                                    0
Studenci do Ludności w wieku 19-24    74
dtype: int64

In [53]:
# Filter the dataframe for rows where 'Studenci do Ludności w wieku 19-24' is zero
zero_values = pp[pp['Studenci do Ludności w wieku 19-24'] == 0]

# Display the corresponding 'Nazwa' values for these rows
print(zero_values[['Kod', 'Nazwa', 'Studenci do Ludności w wieku 19-24']])

              Kod                 Nazwa  Studenci do Ludności w wieku 19-24
175   30210400000  PODREGION WROCŁAWSKI                                 0.0
540   30210400000  PODREGION WROCŁAWSKI                                 0.0
557   51011500000      PODREGION ŁÓDZKI                                 0.0
613   30210400000  PODREGION WROCŁAWSKI                                 0.0
630   51011500000      PODREGION ŁÓDZKI                                 0.0
686   30210400000  PODREGION WROCŁAWSKI                                 0.0
703   51011500000      PODREGION ŁÓDZKI                                 0.0
743   12415100000       PODREGION TYSKI                                 0.0
759   30210400000  PODREGION WROCŁAWSKI                                 0.0
776   51011500000      PODREGION ŁÓDZKI                                 0.0
816   12415100000       PODREGION TYSKI                                 0.0
832   30210400000  PODREGION WROCŁAWSKI                                 0.0
849   510115

In [54]:
# Count NaNs in 'Studenci do Ludności w wieku 19-24'
nan_count = one_factor['Studenci do Ludności w wieku 19-24'].isna().sum()
print(f"Number of NaNs in 'Studenci do Ludności w wieku 19-24': {nan_count}")

# Count zeros in 'Studenci do Ludności w wieku 19-24'
zero_count = (one_factor['Studenci do Ludności w wieku 19-24'] == 0).sum()
print(f"Number of zeros in 'Studenci do Ludności w wieku 19-24': {zero_count}")

Number of NaNs in 'Studenci do Ludności w wieku 19-24': 74
Number of zeros in 'Studenci do Ludności w wieku 19-24': 23


In [55]:
# Filter the dataframe to find rows where 'Studenci do Ludności w wieku 19-24' is NaN
nan_rows = one_factor[one_factor['Studenci do Ludności w wieku 19-24'].isna()]

nan_rows

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Studenci do Ludności w wieku 19-24
0,0,PL213,2007,756.3,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN
14,14,PL214,2007,665.9,228.6,5500,22,3673.14,13898.04,5918.59,8900,36,3205.86,12130,11212000000,NaN
28,28,PL217,2007,459.6,159.2,5400,22,2472.17,9353.97,3983.46,8700,35,2157.68,8164,11212400000,NaN
42,42,PL218,2007,512.7,185.1,5200,21,2685.66,10161.73,4327.45,8400,34,2344.00,8869,11212200000,NaN
56,56,PL219,2007,330.1,110.6,4800,20,1592.80,6026.69,2566.51,7800,32,1390.17,5260,11216900000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,952,PL922,2007,342.3,122.5,6100,25,2088.21,7901.15,3364.77,9800,40,1822.55,6896,71422500000,NaN
980,966,PL923,2007,334.3,137.6,11000,45,3661.63,13854.50,5900.05,17600,72,3195.81,12092,71427000000,NaN
994,980,PL924,2007,383.8,134.1,5700,23,2203.28,8336.53,3550.18,9300,38,1922.99,7276,71422600000,NaN
1008,994,PL925,2007,420.3,164.3,5900,24,2467.03,9334.49,3975.17,9500,39,2153.18,8147,71427100000,NaN


Missing values are for 2007 and there are 0.0 values for 3 regions that contribute to thhose 73 missing values.

#### Factor 2 - population structure by gender 2014 - 2022

Od roku 2020 bazą wyjściową bilansu stanu i struktury ludności są wyniki NSP 2021. W związku z tym dane jak i wskaźniki odnoszące się do liczby i struktury ludności (płeć, grupy wieku) od 2020 roku zostały przeliczone zgodnie z bilansem przygotowanym w oparciu o wyniki NSP 2021. Bilanse liczby i struktury ludności w gminach imiennie opracowane w oparciu o wyniki Narodowych Spisów Powszechnych z uwzględnieniem zmian spowodowanych ruchem naturalnym (urodzenia i zgony), migracjami ludności (na pobyt stały i czasowy) oraz przemieszczeniami związanymi ze zmianami administracyjnymi. Dane o ludności w miejscowościach na podstawie rejestru PESEL. Dane o zarejestrowanych małżeństwach, urodzeniach i zgonach pochodzące ze sprawozdawczości urzędów stanu cywilnego. Dane o orzeczonych rozwodach i separacjach pochodzące ze sprawozdawczości sądów. Dane o migracjach wewnętrznych i zagranicznych na pobyt stały pochodzą z Ministerstwa Spraw Wewnętrznych i Administracji. Prognoza ludności na podstawie badania GUS.

In [56]:
file_path = 'LUDN_2730_XTAB_20231125152006.xlsx'

# Read the specific sheet 'TABLICA'
tablica_df = pd.read_excel(file_path, sheet_name='TABLICA')

tablica_df = tablica_df.copy()

In [57]:
# Cleaning the dataset
men_columns = tablica_df.columns[4:12]  # Starting from 2014
women_columns = tablica_df.columns[15:23]  # Starting from 2014
years = tablica_df.iloc[2, 4:12].values  # Getting the years starting from 2014

# Create a DataFrame for men
df_men = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=men_columns, var_name='TIME_PERIOD', value_name='mężczyźni')
df_men['TIME_PERIOD'] = df_men['TIME_PERIOD'].map(dict(zip(men_columns, years)))

# Create a DataFrame for women
df_women = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=women_columns, var_name='TIME_PERIOD', value_name='kobiety')
df_women['TIME_PERIOD'] = df_women['TIME_PERIOD'].map(dict(zip(women_columns, years)))

# Merge the men and women DataFrames
final_df = pd.merge(df_men, df_women, on=['Kod', 'Nazwa', 'TIME_PERIOD'])

# Select only the relevant columns
final_df = final_df[['Kod', 'Nazwa', 'mężczyźni', 'kobiety', 'TIME_PERIOD']]

final_df

/var/folders/vn/7p3t4_3n02gcv591cgws1gw80000gn/T/ipykernel_18152/3348879281.py:7: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df_men = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=men_columns, var_name='TIME_PERIOD', value_name='mężczyźni')
/var/folders/vn/7p3t4_3n02gcv591cgws1gw80000gn/T/ipykernel_18152/3348879281.py:11: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df_women = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=women_columns, var_name='TIME_PERIOD', value_name='kobiety')


,Kod,Nazwa,mężczyźni,kobiety,TIME_PERIOD
0,NaN,NaN,NaN,NaN,2014
1,NaN,NaN,NaN,NaN,2014
2,NaN,NaN,NaN,2014,2014
3,NaN,NaN,NaN,[lata],2014
4,NaN,NaN,NaN,NaN,2014
...,...,...,...,...,...
707,7.142260e+10,PODREGION OSTROŁĘCKI,70.1,80.1,2021
708,7.142270e+10,PODREGION RADOMSKI,70.7,79.8,2021
709,7.142700e+10,PODREGION PŁOCKI,70.4,78.8,2021
710,7.142710e+10,PODREGION SIEDLECKI,70.4,79.6,2021


In [58]:
# Drop rows where 'Kod' is NaN
final_df = final_df.dropna(subset=['Kod'])

final_df

,Kod,Nazwa,mężczyźni,kobiety,TIME_PERIOD
16,1.121200e+10,PODREGION KRAKOWSKI,74.97,82.12,2014
17,1.121210e+10,PODREGION MIASTO KRAKÓW,76.75,82.45,2014
18,1.121220e+10,PODREGION NOWOSĄDECKI,74.99,82.37,2014
19,1.121230e+10,PODREGION OŚWIĘCIMSKI,73.97,82.22,2014
20,1.121240e+10,PODREGION TARNOWSKI,75.67,82.86,2014
...,...,...,...,...,...
707,7.142260e+10,PODREGION OSTROŁĘCKI,70.1,80.1,2021
708,7.142270e+10,PODREGION RADOMSKI,70.7,79.8,2021
709,7.142700e+10,PODREGION PŁOCKI,70.4,78.8,2021
710,7.142710e+10,PODREGION SIEDLECKI,70.4,79.6,2021


In [59]:
# Rename columns
final_df = final_df.rename(columns={'mężczyźni': 'population men', 'kobiety': 'population women'})
final_df['Kod'] = final_df['Kod'].astype(int).astype(str)

# Convert 'Kod' and 'TIME_PERIOD' to strings to ensure matching types for merge
final_df['Kod'] = final_df['Kod'].astype(str)
one_factor['Kod'] = one_factor['Kod'].astype(str)
final_df['TIME_PERIOD'] = final_df['TIME_PERIOD'].astype(str)
one_factor['TIME_PERIOD'] = one_factor['TIME_PERIOD'].astype(str)

# First, convert 'Kod' to float, then to integer, and finally to string
final_df['Kod'] = final_df['Kod'].astype(float).astype(int).astype(str)

# Ensure 'Kod' in one_factor is also in string format
one_factor['Kod'] = one_factor['Kod'].astype(str)

# Perform the merge
second_factor = pd.merge(one_factor, final_df[['Kod', 'TIME_PERIOD', 'population men', 'population women']], on=['Kod', 'TIME_PERIOD'], how='left')

second_factor

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Studenci do Ludności w wieku 19-24,population men,population women
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN,NaN,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000,255.5,NaN,NaN
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000,272.2,NaN,NaN
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000,254.7,NaN,NaN
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000,272.8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000,3.8,0,0
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000,5.5,0,0
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000,6.7,71.9,80.8
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000,2.1,72.2,80.6


In [60]:
# Count the total number of nulls in each column of 'one_factor'
null_counts = second_factor.isna().sum()

# Display the count of nulls for each column
null_counts

Unnamed: 0                              0
NUTS3 CODE                              0
TIME_PERIOD                             0
POP (THS)                               0
EMP (THS)                               0
EUR_HAB                                 0
EUR_HAB_EU27_2020                       0
MIO_EUR                                 0
MIO_NAC                                 0
MIO_PPS_EU27_2020                       0
PPS_EU27_2020_HAB                       0
PPS_HAB_EU27_2020                       0
GVA (MIO_EUR)                           0
GVA (MIO_NAC)                           0
Kod                                     0
Studenci do Ludności w wieku 19-24     74
population men                        518
population women                      518
dtype: int64

In [61]:
# Filter the dataframe to find rows NaNa 
nan_rows = second_factor[second_factor['population women'].isna()]

nan_rows['TIME_PERIOD']

0       2007
1       2008
2       2009
3       2010
4       2011
        ... 
1024    2009
1025    2010
1026    2011
1027    2012
1028    2013
Name: TIME_PERIOD, Length: 518, dtype: object

Misssing values are for 2007 - 2013 that contribute to this high values of 511 being nans.

### 3. Factor 

In [62]:
f3 = pd.read_excel('WYNA_2497_XTAB_20231123194522.xlsx')
f3

,Kod,Nazwa,2002,2003,2004,2005,2006,2007,2008,2009,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,11212000000,PODREGION KRAKOWSKI,2005.98,2043.51,2091.61,2158.55,2244.29,2472.93,2700.24,2853.76,...,3436.01,3552.25,3775.55,4009.95,4293.01,4622.58,4907.62,5190.51,5321.38,6057.05
1,11212100000,PODREGION MIASTO KRAKÓW,2300.64,2324.15,2463.26,2579.09,2753.21,2995.18,3259.63,3423.56,...,3997.80,4152.52,4431.17,4635.26,4966.20,5368.39,5878.79,6482.24,7203.41,8157.43
2,11212200000,PODREGION NOWOSĄDECKI,1813.69,1895.88,1934.66,2014.91,2086.05,2283.30,2479.05,2601.94,...,2982.13,3077.10,3188.12,3293.79,3472.91,3720.42,3964.93,4260.25,4672.68,5234.58
3,11212300000,PODREGION OŚWIĘCIMSKI,1923.59,2005.17,2135.88,2182.59,2292.52,2503.35,2709.07,2847.04,...,3269.93,3384.20,3488.71,3637.14,3861.09,4146.85,4460.97,4773.53,5216.18,5762.88
4,11212400000,PODREGION TARNOWSKI,1757.93,1852.60,1916.03,1960.45,2085.42,2299.90,2529.45,2697.36,...,3177.39,3264.92,3392.08,3502.00,3670.53,3879.02,4622.36,5009.76,5400.26,5994.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,71422600000,PODREGION OSTROŁĘCKI,1951.67,2005.94,2102.36,2157.11,2276.55,2471.39,2694.14,2880.44,...,3400.61,3543.29,3586.70,3733.47,3952.04,4175.95,4444.58,4809.06,5218.29,5871.61
69,71422700000,PODREGION RADOMSKI,2017.96,2078.24,2138.75,2194.24,2269.29,2469.94,2732.60,2911.12,...,3466.95,3580.06,3667.17,3788.20,4029.81,4266.89,4546.40,4852.53,5441.53,6017.31
70,71427000000,PODREGION PŁOCKI,2461.48,2526.25,2592.58,2686.65,2816.19,3126.02,3408.41,3544.66,...,4227.84,4327.54,4467.57,4595.35,4791.24,5083.15,5437.92,5829.84,6408.11,7120.05
71,71427100000,PODREGION SIEDLECKI,1944.77,2011.01,2091.84,2130.93,2228.81,2397.42,2643.58,2793.20,...,3269.56,3413.81,3562.44,3646.06,3830.06,4097.88,4421.48,4825.59,5232.74,5777.47


In [63]:
# List of year columns to be melted (from 2008 to 2023)
year_columns = [str(year) for year in range(2008, 2023)]

# Melting the DataFrame
melted_f3 = f3.melt(id_vars=['Kod', 'Nazwa'], value_vars=year_columns, var_name='TIME_PERIOD', value_name='Mean_salary')

# Display the transformed DataFrame
melted_f3.isnull().sum()

Kod            0
Nazwa          0
TIME_PERIOD    0
Mean_salary    0
dtype: int64

In [64]:
# Convert 'Kod' and 'TIME_PERIOD' to strings to ensure matching types for merge
melted_f3['Kod'] = melted_f3['Kod'].astype(str)
second_factor['Kod'] = second_factor['Kod'].astype(str)
melted_f3['TIME_PERIOD'] = melted_f3['TIME_PERIOD'].astype(str)
second_factor['TIME_PERIOD'] = second_factor['TIME_PERIOD'].astype(str)

# First, convert 'Kod' to float, then to integer, and finally to string
melted_f3['Kod'] = melted_f3['Kod'].astype(float).astype(int).astype(str)

# Ensure 'Kod' in one_factor is also in string format
second_factor['Kod'] = second_factor['Kod'].astype(str)

# Perform the merge
third_factor = pd.merge(second_factor, melted_f3[['Kod', 'TIME_PERIOD', 'Mean_salary']], on=['Kod', 'TIME_PERIOD'], how='left')

third_factor

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Studenci do Ludności w wieku 19-24,population men,population women,Mean_salary
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN,NaN,NaN,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000,255.5,NaN,NaN,3259.63
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000,272.2,NaN,NaN,3423.56
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000,254.7,NaN,NaN,3543.43
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000,272.8,NaN,NaN,3722.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000,3.8,0,0,4111.52
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000,5.5,0,0,4341.10
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000,6.7,71.9,80.8,4582.52
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000,2.1,72.2,80.6,4902.14


In [65]:
third_factor.isnull().sum()

Unnamed: 0                              0
NUTS3 CODE                              0
TIME_PERIOD                             0
POP (THS)                               0
EMP (THS)                               0
EUR_HAB                                 0
EUR_HAB_EU27_2020                       0
MIO_EUR                                 0
MIO_NAC                                 0
MIO_PPS_EU27_2020                       0
PPS_EU27_2020_HAB                       0
PPS_HAB_EU27_2020                       0
GVA (MIO_EUR)                           0
GVA (MIO_NAC)                           0
Kod                                     0
Studenci do Ludności w wieku 19-24     74
population men                        518
population women                      518
Mean_salary                            74
dtype: int64

In [66]:
# Filter the dataframe
nan_rows = third_factor[third_factor['Mean_salary'].isna()]

nan_rows['TIME_PERIOD'].unique()

array(['2007'], dtype=object)

Missinbg values for 2007 year that contribute to 73 empty rows

### 4. Factor - Studenci ogółem 2012 - 2020

In [67]:
f4 = pd.read_csv('SZKO_3226_CTAB_20231125151701.csv', delimiter = ';')
f4 = f4.copy()
f4

,Kod,Nazwa,studenci ogółem;2012;[osoba],studenci ogółem;2013;[osoba],studenci ogółem;2014;[osoba],studenci ogółem;2015;[osoba],studenci ogółem;2016;[osoba],studenci ogółem;2017;[osoba],studenci ogółem;2018;[osoba],studenci ogółem;2019;[osoba],studenci ogółem;2020;[osoba],studenci ogółem;2021;[osoba],studenci ogółem;2022;[osoba],Unnamed: 13
0,11212000000,PODREGION KRAKOWSKI,1338,1423,1432,1216,1117,113,1100,1162,1206,1158,1205,NaN
1,11212100000,PODREGION MIASTO KRAKÓW,181924,170519,165857,160808,154332,143613,134969,129887,130428,130360,129357,NaN
2,11212200000,PODREGION NOWOSĄDECKI,7410,6434,5592,5027,4547,4542,4450,4619,5399,6471,8511,NaN
3,11212300000,PODREGION OŚWIĘCIMSKI,2151,1839,1824,1588,1568,1764,1961,1969,2133,2018,1917,NaN
4,11212400000,PODREGION TARNOWSKI,7127,6566,6016,5397,5045,5178,5084,5035,4956,5038,4627,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,71422600000,PODREGION OSTROŁĘCKI,1587,1177,1013,822,676,619,553,558,622,693,702,NaN
69,71422700000,PODREGION RADOMSKI,13257,11808,10654,9844,8856,7621,6990,7219,7028,7561,7926,NaN
70,71427000000,PODREGION PŁOCKI,7943,7296,6887,6363,5990,5777,5345,4994,4578,4235,3933,NaN
71,71427100000,PODREGION SIEDLECKI,10727,9506,8705,7419,6586,6446,6257,6252,6298,6245,6008,NaN


In [68]:
# Selecting the columns for unpivoting
id_vars = ['Kod', 'Nazwa']
value_vars = [col for col in f4.columns if col.startswith('studenci ogółem')]

# Unpivot the DataFrame
f4 = f4.melt(id_vars=id_vars, value_vars=value_vars, var_name='TIME_PERIOD', value_name='Studenci')

# Extract the year from the 'TIME_PERIOD' column
f4['TIME_PERIOD'] = f4['TIME_PERIOD'].str.extract('(\d{4})')

print(f4.head())

           Kod                    Nazwa TIME_PERIOD  Studenci
0  11212000000      PODREGION KRAKOWSKI        2012      1338
1  11212100000  PODREGION MIASTO KRAKÓW        2012    181924
2  11212200000    PODREGION NOWOSĄDECKI        2012      7410
3  11212300000    PODREGION OŚWIĘCIMSKI        2012      2151
4  11212400000      PODREGION TARNOWSKI        2012      7127


In [69]:
# Convert 'Kod' and 'TIME_PERIOD' to strings to ensure matching types for merge
f4['Kod'] = f4['Kod'].astype(str)

# First, convert 'Kod' to float, then to integer, and finally to string
#f4['Kod'] = f4['Kod'].astype(float).astype(int).astype(str)

# Ensure 'Kod' in one_factor is also in string format
third_factor['Kod'] = third_factor['Kod'].astype(str)

# Perform the merge
forth_factor = pd.merge(third_factor, f4[['Kod', 'TIME_PERIOD', 'Studenci']], on=['Kod', 'TIME_PERIOD'], how='left')

forth_factor

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,PPS_EU27_2020_HAB,PPS_HAB_EU27_2020,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Studenci do Ludności w wieku 19-24,population men,population women,Mean_salary,Studenci
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,21300,87,8712.37,32965,11212100000,NaN,NaN,NaN,NaN,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,22300,88,9939.92,34910,11212100000,255.5,NaN,NaN,3259.63,NaN
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,22400,93,8659.07,37473,11212100000,272.2,NaN,NaN,3423.56,NaN
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,24400,98,9594.71,38328,11212100000,254.7,NaN,NaN,3543.43,NaN
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,26000,101,10093.43,41591,11212100000,272.8,NaN,NaN,3722.48,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,17200,61,2272.64,9916,71427300000,3.8,0,0,4111.52,718.0
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,17700,60,2425.65,10326,71427300000,5.5,0,0,4341.10,990.0
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,18200,60,2502.64,10665,71427300000,6.7,71.9,80.8,4582.52,1146.0
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,19300,62,2671.96,11483,71427300000,2.1,72.2,80.6,4902.14,344.0


In [70]:
forth_factor.isnull().sum()

Unnamed: 0                              0
NUTS3 CODE                              0
TIME_PERIOD                             0
POP (THS)                               0
EMP (THS)                               0
EUR_HAB                                 0
EUR_HAB_EU27_2020                       0
MIO_EUR                                 0
MIO_NAC                                 0
MIO_PPS_EU27_2020                       0
PPS_EU27_2020_HAB                       0
PPS_HAB_EU27_2020                       0
GVA (MIO_EUR)                           0
GVA (MIO_NAC)                           0
Kod                                     0
Studenci do Ludności w wieku 19-24     74
population men                        518
population women                      518
Mean_salary                            74
Studenci                              370
dtype: int64

In [71]:
# Filter the dataframe to find NaNs rows
nan_rows = forth_factor[forth_factor['Studenci'].isna()]

nan_rows['TIME_PERIOD'].unique()

array(['2007', '2008', '2009', '2010', '2011'], dtype=object)

Missing values for 2007 - 2011

### Factor 5 - Ludność w miastach w % ogółu ludności (dane półroczne) 2012 - 2020

In [72]:
file_path = 'LUDN_2463_XTAB_20231125152108.xlsx'

# Read the specific sheet 'TABLICA'
tablica_df = pd.read_excel(file_path, sheet_name='TABLICA')

# Display the first few rows of the DataFrame 
tablica_df = tablica_df.copy()
tablica_df

,Kod,Nazwa,rok,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,NaN,NaN,[%],[%],[%],[%],[%],[%],[%],[%],[%],[%],[%]
2,1.121200e+10,PODREGION KRAKOWSKI,22.86,22.78,22.72,22.67,22.65,22.61,22.57,22.61,22.2,22.12,22.05
3,1.121210e+10,PODREGION MIASTO KRAKÓW,100,100,100,100,100,100,100,100,100,100,100
4,1.121220e+10,PODREGION NOWOSĄDECKI,34.06,33.88,33.69,33.56,33.42,33.27,33.07,32.93,32.4,32.17,31.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,7.142260e+10,PODREGION OSTROŁĘCKI,36.34,36.23,36.22,36.27,36.25,36.21,36.27,36.27,36.01,35.89,35.77
71,7.142270e+10,PODREGION RADOMSKI,49.36,49.2,49.04,48.92,48.77,48.71,48.52,48.36,47.57,47.51,47.95
72,7.142700e+10,PODREGION PŁOCKI,51.21,51.02,50.92,50.86,50.77,50.7,51.22,51.16,50.64,50.5,50.29
73,7.142710e+10,PODREGION SIEDLECKI,36.95,37.03,37.15,37.31,37.39,37.58,37.7,37.81,37.83,37.9,37.9


In [73]:
# Get the 'rok' column data for the year 2012
rok_2012 = tablica_df[['Kod', 'Nazwa', 'rok']].copy()
rok_2012['TIME_PERIOD'] = 2012
rok_2012.rename(columns={'rok': 'population in cities in %'}, inplace=True)

# List of year columns to be melted (from 2013 to 2022)
year_columns = [f'Unnamed: {i}' for i in range(3, 13)]

# Melting the DataFrame for years 2013 to 2022
melted_tablica_df = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=year_columns, var_name='TIME_PERIOD', value_name='population in cities in %')

# Adjust TIME_PERIOD values in melted_tablica_df to correspond to years
years = [str(year) for year in range(2013, 2023)]
melted_tablica_df['TIME_PERIOD'] = melted_tablica_df['TIME_PERIOD'].map(dict(zip(year_columns, years)))

# Concatenate the data for 2012 with the rest
final_tablica_df = pd.concat([rok_2012, melted_tablica_df])

# Remove rows with NaN values in 'Kod' and 'Nazwa' columns
final_tablica_df = final_tablica_df.dropna(subset=['Kod', 'Nazwa'])
final_tablica_df.isnull().sum()

Kod                          0
Nazwa                        0
population in cities in %    0
TIME_PERIOD                  0
dtype: int64

In [74]:
# Ensure 'TIME_PERIOD' is in the correct format
final_tablica_df['TIME_PERIOD'] = final_tablica_df['TIME_PERIOD'].astype(str)
forth_factor['TIME_PERIOD'] = forth_factor['TIME_PERIOD'].astype(str)

# Ensure 'Kod' is in the correct format
final_tablica_df['Kod'] = final_tablica_df['Kod'].astype(float).astype(int).astype(str)
forth_factor['Kod'] = forth_factor['Kod'].astype(str).str.strip()

# Perform the merge
fifth_factor = pd.merge(forth_factor, final_tablica_df[['Kod', 'TIME_PERIOD', 'population in cities in %']], on=['Kod', 'TIME_PERIOD'], how='left')

In [75]:
fifth_factor.isnull().sum()

Unnamed: 0                              0
NUTS3 CODE                              0
TIME_PERIOD                             0
POP (THS)                               0
EMP (THS)                               0
EUR_HAB                                 0
EUR_HAB_EU27_2020                       0
MIO_EUR                                 0
MIO_NAC                                 0
MIO_PPS_EU27_2020                       0
PPS_EU27_2020_HAB                       0
PPS_HAB_EU27_2020                       0
GVA (MIO_EUR)                           0
GVA (MIO_NAC)                           0
Kod                                     0
Studenci do Ludności w wieku 19-24     74
population men                        518
population women                      518
Mean_salary                            74
Studenci                              370
population in cities in %             370
dtype: int64

In [76]:
# Filter the dataframe to find NaNs rows
nan_rows = fifth_factor[fifth_factor['population in cities in %'].isna()]

nan_rows['TIME_PERIOD'].unique()

array(['2007', '2008', '2009', '2010', '2011'], dtype=object)

Missing values for 2007 - 2011

### Factor 6 - Number of students at technological universities 2008 - 2018 

In [77]:
tech_df = pd.read_csv('SZKO_2133_CTAB_20231123195949.csv',delimiter = ';')
tech_df.head()

,Kod,Nazwa,uczelnie techniczne;studenci;ogółem;2008;[osoba],uczelnie techniczne;studenci;ogółem;2009;[osoba],uczelnie techniczne;studenci;ogółem;2010;[osoba],uczelnie techniczne;studenci;ogółem;2011;[osoba],uczelnie techniczne;studenci;ogółem;2012;[osoba],uczelnie techniczne;studenci;ogółem;2013;[osoba],uczelnie techniczne;studenci;ogółem;2014;[osoba],uczelnie techniczne;studenci;ogółem;2015;[osoba],uczelnie techniczne;studenci;ogółem;2016;[osoba],uczelnie techniczne;studenci;ogółem;2017;[osoba],uczelnie techniczne;studenci;ogółem;2018;[osoba],Unnamed: 13
0,11212100000,PODREGION MIASTO KRAKÓW,45461,46251,48352,50035,52052,50058,48389,45451,42942,39927,36775,NaN
1,11212200000,PODREGION NOWOSĄDECKI,198,146,139,129,136,84,0,0,0,0,0,NaN
2,11212300000,PODREGION OŚWIĘCIMSKI,0,78,62,0,0,0,0,0,0,0,0,NaN
3,12414400000,PODREGION BIELSKI,7955,7613,7390,7282,7038,6690,6446,5826,5482,4813,4442,NaN
4,12414500000,PODREGION BYTOMSKI,181,200,659,483,211,37,0,0,0,0,0,NaN


In [78]:
# Columns to be melted
year_columns = [col for col in tech_df.columns if 'uczelnie techniczne;studenci;ogółem;' in col]

# Melting the DataFrame
melted_tech_df = tech_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=year_columns, var_name='TIME_PERIOD', value_name='Tech Universities; students')

# Extracting the year from 'TIME_PERIOD' column
melted_tech_df['TIME_PERIOD'] = melted_tech_df['TIME_PERIOD'].str.extract('(\d{4})')

melted_tech_df

,Kod,Nazwa,TIME_PERIOD,Tech Universities; students
0,11212100000,PODREGION MIASTO KRAKÓW,2008,45461
1,11212200000,PODREGION NOWOSĄDECKI,2008,198
2,11212300000,PODREGION OŚWIĘCIMSKI,2008,0
3,12414400000,PODREGION BIELSKI,2008,7955
4,12414500000,PODREGION BYTOMSKI,2008,181
...,...,...,...,...
479,62013900000,PODREGION SUWALSKI,2018,0
480,71412800000,PODREGION MIASTO WARSZAWA,2018,35544
481,71422700000,PODREGION RADOMSKI,2018,4056
482,71427000000,PODREGION PŁOCKI,2018,1586


In [79]:
# Ensure 'TIME_PERIOD' is in the correct format
melted_tech_df['TIME_PERIOD'] = melted_tech_df['TIME_PERIOD'].astype(str)
fifth_factor['TIME_PERIOD'] = fifth_factor['TIME_PERIOD'].astype(str)

# Ensure 'Kod' is in the correct format
melted_tech_df['Kod'] = melted_tech_df['Kod'].astype(float).astype(int).astype(str)
fifth_factor['Kod'] = fifth_factor['Kod'].astype(str).str.strip()

sixth_factor = pd.merge(fifth_factor, melted_tech_df[['Kod', 'TIME_PERIOD', 'Tech Universities; students']], on=['Kod', 'TIME_PERIOD'], how='left')
sixth_factor

,Unnamed: 0,NUTS3 CODE,TIME_PERIOD,POP (THS),EMP (THS),EUR_HAB,EUR_HAB_EU27_2020,MIO_EUR,MIO_NAC,MIO_PPS_EU27_2020,...,GVA (MIO_EUR),GVA (MIO_NAC),Kod,Studenci do Ludności w wieku 19-24,population men,population women,Mean_salary,Studenci,population in cities in %,Tech Universities; students
0,0,PL213,2007,756.30,428.6,13200,54,9982.27,37769.90,16084.62,...,8712.37,32965,11212100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,PL213,2008,756.40,428.6,15100,60,11393.55,40015.30,16853.02,...,9939.92,34910,11212100000,255.5,NaN,NaN,3259.63,NaN,NaN,45461.0
2,2,PL213,2009,761.42,428.6,12800,53,9755.34,42217.21,17053.12,...,8659.07,37473,11212100000,272.2,NaN,NaN,3423.56,NaN,NaN,46251.0
3,3,PL213,2010,757.11,428.6,14600,58,11017.52,44011.69,18453.85,...,9594.71,38328,11212100000,254.7,NaN,NaN,3543.43,NaN,NaN,48352.0
4,4,PL213,2011,758.21,433.6,15300,60,11581.62,47723.22,19738.61,...,10093.43,41591,11212100000,272.8,NaN,NaN,3722.48,NaN,NaN,50035.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1017,PL926,2016,259.80,108.3,9800,35,2556.74,11155.56,4471.49,...,2272.64,9916,71427300000,3.8,0,0,4111.52,718.0,45.59,0.0
1032,1018,PL926,2017,259.70,107.8,10500,36,2739.42,11661.73,4595.52,...,2425.65,10326,71427300000,5.5,0,0,4341.10,990.0,45.46,0.0
1033,1019,PL926,2018,259.80,107.4,11000,36,2857.49,12177.18,4722.54,...,2502.64,10665,71427300000,6.7,71.9,80.8,4582.52,1146.0,45.3,0.0
1034,1020,PL926,2019,259.10,107.1,11700,37,3040.11,13065.19,5001.82,...,2671.96,11483,71427300000,2.1,72.2,80.6,4902.14,344.0,45.2,NaN


In [80]:
# Read the specific sheet 'TABLICA'
tablica_df = pd.read_csv('LUDN_2730_CTAB_20231125151959.csv', delimiter = ';')

# Display the first few rows of the DataFrame 
tablica_df = tablica_df.copy()
tablica_df

,Kod,Nazwa,mężczyźni;ogółem;0;2012;[lata],mężczyźni;ogółem;0;2013;[lata],mężczyźni;ogółem;0;2014;[lata],mężczyźni;ogółem;0;2015;[lata],mężczyźni;ogółem;0;2016;[lata],mężczyźni;ogółem;0;2017;[lata],mężczyźni;ogółem;0;2018;[lata],mężczyźni;ogółem;0;2019;[lata],...,kobiety;ogółem;0;2014;[lata],kobiety;ogółem;0;2015;[lata],kobiety;ogółem;0;2016;[lata],kobiety;ogółem;0;2017;[lata],kobiety;ogółem;0;2018;[lata],kobiety;ogółem;0;2019;[lata],kobiety;ogółem;0;2020;[lata],kobiety;ogółem;0;2021;[lata],kobiety;ogółem;0;2022;[lata],Unnamed: 24
0,11212000000,PODREGION KRAKOWSKI,"0,0","0,0","75,0","74,5","75,0","74,6","74,8","74,9",...,"82,1","82,0","82,5","82,3","82,7","82,6","81,5","81,1","82,3",NaN
1,11212100000,PODREGION MIASTO KRAKÓW,"0,0","0,0","76,8","76,3","77,1","76,8","76,7","76,5",...,"82,5","82,6","83,1","82,7","83,2","83,0","82,1","81,8","82,3",NaN
2,11212200000,PODREGION NOWOSĄDECKI,"0,0","0,0","75,0","75,2","74,7","75,2","75,8","75,5",...,"82,4","82,4","82,8","83,0","83,0","82,6","81,6","80,6","82,6",NaN
3,11212300000,PODREGION OŚWIĘCIMSKI,"0,0","0,0","74,0","74,3","74,4","74,4","73,9","74,2",...,"82,2","81,8","82,2","82,0","82,1","82,0","80,8","79,9","81,2",NaN
4,11212400000,PODREGION TARNOWSKI,"0,0","0,0","75,7","75,4","75,6","75,7","75,8","75,6",...,"82,9","82,8","83,7","82,7","83,4","82,9","81,7","81,0","82,5",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,71422600000,PODREGION OSTROŁĘCKI,"0,0","0,0","73,0","72,2","73,2","72,4","72,9","73,1",...,"81,7","82,3","82,5","82,3","82,3","81,5","80,3","80,1","81,3",NaN
69,71422700000,PODREGION RADOMSKI,"0,0","0,0","72,4","72,5","72,5","72,6","72,2","72,7",...,"81,2","81,4","81,9","81,8","81,8","81,9","80,8","79,8","81,5",NaN
70,71427000000,PODREGION PŁOCKI,"0,0","0,0","72,2","72,3","72,0","72,9","72,2","73,0",...,"80,4","81,3","81,0","80,7","80,7","81,0","79,3","78,8","80,1",NaN
71,71427100000,PODREGION SIEDLECKI,"0,0","0,0","0,0","0,0","0,0","0,0","73,5","72,8",...,"0,0","0,0","0,0","0,0","82,0","82,2","80,7","79,6","80,7",NaN


In [81]:
# Columns for men and women
men_columns = [col for col in tablica_df.columns if 'mężczyźni;ogółem' in col]
women_columns = [col for col in tablica_df.columns if 'kobiety;ogółem' in col]

# Melting the DataFrame for men
melted_men = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=men_columns, var_name='TIME_PERIOD', value_name='Men')
melted_men['TIME_PERIOD'] = melted_men['TIME_PERIOD'].str.extract('(\d{4})')

# Melting the DataFrame for women
melted_women = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=women_columns, var_name='TIME_PERIOD', value_name='Women')
melted_women['TIME_PERIOD'] = melted_women['TIME_PERIOD'].str.extract('(\d{4})')

# Combine the melted DataFrames
combined_df = pd.merge(melted_men, melted_women, on=['Kod', 'Nazwa', 'TIME_PERIOD'])

combined_df

,Kod,Nazwa,TIME_PERIOD,Men,Women
0,11212000000,PODREGION KRAKOWSKI,2012,"0,0","0,0"
1,11212100000,PODREGION MIASTO KRAKÓW,2012,"0,0","0,0"
2,11212200000,PODREGION NOWOSĄDECKI,2012,"0,0","0,0"
3,11212300000,PODREGION OŚWIĘCIMSKI,2012,"0,0","0,0"
4,11212400000,PODREGION TARNOWSKI,2012,"0,0","0,0"
...,...,...,...,...,...
798,71422600000,PODREGION OSTROŁĘCKI,2022,"72,0","81,3"
799,71422700000,PODREGION RADOMSKI,2022,"72,9","81,5"
800,71427000000,PODREGION PŁOCKI,2022,"71,8","80,1"
801,71427100000,PODREGION SIEDLECKI,2022,"72,3","80,7"


In [82]:
# Cleaning the dataset
men_columns = tablica_df.columns[4:12]  # Starting from 2014
women_columns = tablica_df.columns[15:23]  # Starting from 2014
years = tablica_df.iloc[2, 4:12].values  # Getting the years starting from 2014

# Create a DataFrame for men
df_men = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=men_columns, var_name='TIME_PERIOD', value_name='mężczyźni')
df_men['TIME_PERIOD'] = df_men['TIME_PERIOD'].map(dict(zip(men_columns, years)))

# Create a DataFrame for women
df_women = tablica_df.melt(id_vars=['Kod', 'Nazwa'], value_vars=women_columns, var_name='TIME_PERIOD', value_name='kobiety')
df_women['TIME_PERIOD'] = df_women['TIME_PERIOD'].map(dict(zip(women_columns, years)))

# Merge the men and women DataFrames
final_df = pd.merge(df_men, df_women, on=['Kod', 'Nazwa', 'TIME_PERIOD'])

# Select only the relevant columns
final_df = final_df[['Kod', 'Nazwa', 'mężczyźni', 'kobiety', 'TIME_PERIOD']]

final_df

,Kod,Nazwa,mężczyźni,kobiety,TIME_PERIOD
0,11212000000,PODREGION KRAKOWSKI,"75,0","82,1","75,0"
1,11212100000,PODREGION MIASTO KRAKÓW,"76,8","82,5","75,0"
2,11212200000,PODREGION NOWOSĄDECKI,"75,0","82,4","75,0"
3,11212300000,PODREGION OŚWIĘCIMSKI,"74,0","82,2","75,0"
4,11212400000,PODREGION TARNOWSKI,"75,7","82,9","75,0"
...,...,...,...,...,...
871,71427100000,PODREGION SIEDLECKI,"70,4","79,6","73,3"
872,71427300000,PODREGION ŻYRARDOWSKI,"70,9","79,3","73,3"
873,71427300000,PODREGION ŻYRARDOWSKI,"70,9","78,0","73,3"
874,71427300000,PODREGION ŻYRARDOWSKI,"69,7","79,3","73,3"


In [83]:
sixth_factor = sixth_factor.to_excel('all_factors_nuts3.xlsx')

####  We've added six new factors at the NUTS 3 level to refine our modeling for Poland. 
A key consideration is the availability of these factors over time. Unlike our existing dataset that covers 2007-2020, these new factors are only available for the years 2014-2020. This aspect needs careful handling in our analysis and modeling process.

### We will also put together factors on the NUTS 2 level for more general overview
Again we will use mapping dataset as a baseline and use them to merge new factors with our Polish datase that doesn"t

### Factor 1 Mejsca Pracy 2008 - 2022
***Nowo utworzone miejsca pracy to miejsca powstałe w wyniku zmian organizacyjnych, rozszerzenia lub zmiany profilu działalności oraz wszystkie miejsca pracy w jednostkach nowo powstałych.***

In [84]:
rynek = pd.read_csv('RYNE_3441_CTAB_20231201123702.csv', delimiter = ';')
rynek

,Kod,Nazwa,nowo utworzone miejsca pracy;2008;[tys. msc.],nowo utworzone miejsca pracy;2009;[tys. msc.],nowo utworzone miejsca pracy;2010;[tys. msc.],nowo utworzone miejsca pracy;2011;[tys. msc.],nowo utworzone miejsca pracy;2012;[tys. msc.],nowo utworzone miejsca pracy;2013;[tys. msc.],nowo utworzone miejsca pracy;2014;[tys. msc.],nowo utworzone miejsca pracy;2015;[tys. msc.],nowo utworzone miejsca pracy;2016;[tys. msc.],nowo utworzone miejsca pracy;2017;[tys. msc.],nowo utworzone miejsca pracy;2018;[tys. msc.],nowo utworzone miejsca pracy;2019;[tys. msc.],nowo utworzone miejsca pracy;2020;[tys. msc.],nowo utworzone miejsca pracy;2021;[tys. msc.],nowo utworzone miejsca pracy;2022;[tys. msc.],Unnamed: 17
0,0,POLSKA,"490,6","521,6","609,3","580,3","465,0","502,4","614,8","595,8","618,7","694,1","717,8","674,8","470,4","582,7","534,8",NaN
1,200000,DOLNOŚLĄSKIE,"43,2","41,1","43,7","38,3","33,2","42,7","44,6","47,9","51,3","57,6","47,6","51,5","45,4","46,9","40,0",NaN
2,400000,KUJAWSKO-POMORSKIE,"23,6","26,7","25,4","16,0","13,2","17,8","25,3","24,2","23,4","25,9","31,2","23,8","20,3","24,1","21,3",NaN
3,600000,LUBELSKIE,"13,3","16,9","22,6","21,5","17,9","18,6","19,7","20,2","25,8","24,6","28,2","27,1","17,9","23,4","19,5",NaN
4,800000,LUBUSKIE,"15,2","17,2","13,6","59,0","48,8","10,8","11,6","12,3","13,1","14,3","11,2","12,3","7,7","12,1","11,9",NaN
5,1000000,ŁÓDZKIE,"31,7","44,7","36,4","40,9","28,5","29,8","37,7","38,7","37,3","41,0","39,5","54,6","29,3","41,1","29,0",NaN
6,1200000,MAŁOPOLSKIE,"35,4","49,5","57,9","44,1","35,8","48,0","48,9","52,8","68,6","66,9","68,9","76,4","49,5","61,8","54,5",NaN
7,1400000,MAZOWIECKIE,"110,9","100,6","123,2","150,2","93,7","96,8","163,0","123,9","122,8","150,5","151,7","135,3","98,0","122,2","126,3",NaN
8,1600000,OPOLSKIE,"11,5","10,8","14,1","13,9","13,4","9,2","12,2","10,0","15,2","13,2","14,0","11,9","10,4","11,0","8,2",NaN
9,1800000,PODKARPACKIE,"16,6","18,4","23,7","23,1","20,3","24,0","20,2","27,7","22,0","30,2","30,5","26,9","21,1","23,3","20,4",NaN


In [85]:
# Dropping the 'Unnamed: 17' column
rynek = rynek.drop(columns=['Unnamed: 17'])

# Reversing the order of the columns for the years 2018 to 2022
years_columns = rynek.columns[2:][::-1]  # Reverse the order of the year columns

# Melting the DataFrame to create 'TIME_PERIOD' and 'Miejsca Pracy' columns
melted_rynek = rynek.melt(id_vars=["Kod", "Nazwa"], 
                          value_vars=years_columns, 
                          var_name="TIME_PERIOD", 
                          value_name="Sites of Worky [tys. places]")

# Extracting the year from 'TIME_PERIOD' column
melted_rynek['TIME_PERIOD'] = melted_rynek['TIME_PERIOD'].str.extract('(\d{4})')

melted_rynek

,Kod,Nazwa,TIME_PERIOD,Sites of Worky [tys. places]
0,0,POLSKA,2022,"534,8"
1,200000,DOLNOŚLĄSKIE,2022,"40,0"
2,400000,KUJAWSKO-POMORSKIE,2022,"21,3"
3,600000,LUBELSKIE,2022,"19,5"
4,800000,LUBUSKIE,2022,"11,9"
...,...,...,...,...
250,2400000,ŚLĄSKIE,2008,"68,9"
251,2600000,ŚWIĘTOKRZYSKIE,2008,"7,2"
252,2800000,WARMIŃSKO-MAZURSKIE,2008,"15,1"
253,3000000,WIELKOPOLSKIE,2008,"51,7"


In [86]:
melted_rynek.isna().sum()

Kod                             0
Nazwa                           0
TIME_PERIOD                     0
Sites of Worky [tys. places]    0
dtype: int64

### Factor 2 Absolwenci uniwerysetów technicznych 2008 - 2018

In [87]:
tech_uni = pd.read_csv('SZKO_2133_CTAB_20231201131942.csv',delimiter = ';')
tech_uni

,Kod,Nazwa,uczelnie techniczne;absolwenci;ogółem;2008;[osoba],uczelnie techniczne;absolwenci;ogółem;2009;[osoba],uczelnie techniczne;absolwenci;ogółem;2010;[osoba],uczelnie techniczne;absolwenci;ogółem;2011;[osoba],uczelnie techniczne;absolwenci;ogółem;2012;[osoba],uczelnie techniczne;absolwenci;ogółem;2013;[osoba],uczelnie techniczne;absolwenci;ogółem;2014;[osoba],uczelnie techniczne;absolwenci;ogółem;2015;[osoba],uczelnie techniczne;absolwenci;ogółem;2016;[osoba],uczelnie techniczne;absolwenci;ogółem;2017;[osoba],uczelnie techniczne;absolwenci;ogółem;2018;[osoba],Unnamed: 13
0,0,POLSKA,53398,52606,59282,74494,77155,78248,78436,79428,77195,82666,69209,NaN
1,200000,DOLNOŚLĄSKIE,4939,4876,4885,7084,7015,7435,7492,7732,8198,7988,7528,NaN
2,400000,KUJAWSKO-POMORSKIE,652,561,707,378,336,249,137,103,66,13,42,NaN
3,600000,LUBELSKIE,1510,1784,1642,2186,2277,2558,2569,2563,2610,2679,2503,NaN
4,800000,LUBUSKIE,0,0,12,14,0,0,2,0,0,0,0,NaN
5,1000000,ŁÓDZKIE,3484,3780,4393,4815,4929,4208,4317,4505,4309,4011,3929,NaN
6,1200000,MAŁOPOLSKIE,6357,6407,6775,11344,12621,12664,12923,13773,13247,14789,12335,NaN
7,1400000,MAZOWIECKIE,7806,8341,8955,9190,9725,10134,10348,9976,9617,10663,9299,NaN
8,1600000,OPOLSKIE,1686,2142,2877,2811,2934,2614,2636,2254,2065,2239,1722,NaN
9,1800000,PODKARPACKIE,2722,2463,3019,3886,3638,4331,4610,5206,5209,4841,4335,NaN


In [88]:
# Dropping the 'Unnamed: 13' column
tech_uni = tech_uni.drop(columns=['Unnamed: 13'])

# Reversing the order of the columns 
years_columns = tech_uni.columns[2:][::-1]  # Reverse the order of the year columns

# Melting the DataFrame to create 'TIME_PERIOD' and 'Miejsca Pracy' columns
melted_tech_uni = tech_uni.melt(id_vars=["Kod", "Nazwa"], 
                          value_vars=years_columns, 
                          var_name="TIME_PERIOD", 
                          value_name="Uni_tech graduates[person.]")

# Extracting the year from 'TIME_PERIOD' column
melted_tech_uni['TIME_PERIOD'] = melted_tech_uni['TIME_PERIOD'].str.extract('(\d{4})')

melted_tech_uni

,Kod,Nazwa,TIME_PERIOD,Uni_tech graduates[person.]
0,0,POLSKA,2018,69209
1,200000,DOLNOŚLĄSKIE,2018,7528
2,400000,KUJAWSKO-POMORSKIE,2018,42
3,600000,LUBELSKIE,2018,2503
4,800000,LUBUSKIE,2018,0
...,...,...,...,...
171,2200000,POMORSKIE,2008,2534
172,2400000,ŚLĄSKIE,2008,10790
173,2600000,ŚWIĘTOKRZYSKIE,2008,1225
174,3000000,WIELKOPOLSKIE,2008,3895


#### Factor 3 Absolwenci uniwerysetów technicznych 2008 - 2018 - Kobiety  

In [89]:
tech_wom = pd.read_csv('SZKO_2134_CTAB_20231201135851.csv', delimiter = ';')
tech_wom

,Kod,Nazwa,uczelnie techniczne;absolwenci;kobiety;2008;[osoba],uczelnie techniczne;absolwenci;kobiety;2009;[osoba],uczelnie techniczne;absolwenci;kobiety;2010;[osoba],uczelnie techniczne;absolwenci;kobiety;2011;[osoba],uczelnie techniczne;absolwenci;kobiety;2012;[osoba],uczelnie techniczne;absolwenci;kobiety;2013;[osoba],uczelnie techniczne;absolwenci;kobiety;2014;[osoba],uczelnie techniczne;absolwenci;kobiety;2015;[osoba],uczelnie techniczne;absolwenci;kobiety;2016;[osoba],uczelnie techniczne;absolwenci;kobiety;2017;[osoba],uczelnie techniczne;absolwenci;kobiety;2018;[osoba],Unnamed: 13
0,0,POLSKA,20557,20269,22571,28442,29976,32390,33063,34090,33441,35612,29682,NaN
1,200000,DOLNOŚLĄSKIE,1543,1567,1533,2220,2310,2630,2781,2925,3193,3174,2914,NaN
2,400000,KUJAWSKO-POMORSKIE,164,174,168,121,92,45,29,21,6,1,6,NaN
3,600000,LUBELSKIE,550,652,516,637,789,953,921,1001,970,972,925,NaN
4,800000,LUBUSKIE,0,0,0,1,0,0,0,0,0,0,0,NaN
5,1000000,ŁÓDZKIE,1454,1577,1702,2054,2070,1945,2038,2151,2060,1954,1918,NaN
6,1200000,MAŁOPOLSKIE,2175,2101,2381,3880,4383,4713,4981,5526,5431,6218,5123,NaN
7,1400000,MAZOWIECKIE,3252,3288,3193,3573,3754,4296,4536,4416,4330,4660,4125,NaN
8,1600000,OPOLSKIE,762,1011,1299,1258,1277,1164,1179,1047,959,938,752,NaN
9,1800000,PODKARPACKIE,1280,1037,1469,1676,1546,2077,2249,2343,2411,2302,2057,NaN


In [90]:
# Dropping the 'Unnamed: 13' column
tech_wom = tech_wom.drop(columns=['Unnamed: 13'])

# Reversing the order of the columns 
years_columns = tech_wom.columns[2:][::-1]  # Reverse the order of the year columns

# Melting the DataFrame to create 'TIME_PERIOD' and 'Miejsca Pracy' columns
melted_tech_wom = tech_wom.melt(id_vars=["Kod", "Nazwa"], 
                          value_vars=years_columns, 
                          var_name="TIME_PERIOD", 
                          value_name="Women - Uni_tech graduates [person.]")

# Extracting the year from 'TIME_PERIOD' column
melted_tech_wom['TIME_PERIOD'] = melted_tech_uni['TIME_PERIOD'].str.extract('(\d{4})')

melted_tech_wom

,Kod,Nazwa,TIME_PERIOD,Women - Uni_tech graduates [person.]
0,0,POLSKA,2018,29682
1,200000,DOLNOŚLĄSKIE,2018,2914
2,400000,KUJAWSKO-POMORSKIE,2018,6
3,600000,LUBELSKIE,2018,925
4,800000,LUBUSKIE,2018,0
...,...,...,...,...
171,2200000,POMORSKIE,2008,1002
172,2400000,ŚLĄSKIE,2008,4482
173,2600000,ŚWIĘTOKRZYSKIE,2008,363
174,3000000,WIELKOPOLSKIE,2008,1100


#### Factor 4 - Employment in technology and knowledge-intensive sectors by NUTS 2 regions and sex 2008 - 2020

In [109]:
empl_in_tech = pd.read_csv('htec_emp_reg2_page_linear.csv', delimiter = ',')
empl_in_tech

,DATAFLOW,LAST UPDATE,freq,nace_r2,unit,sex,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL2,2008,67.9,b
1,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL2,2009,71.7,NaN
2,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL2,2010,67.4,b
3,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL2,2011,76.0,NaN
4,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL2,2012,86.7,NaN
...,...,...,...,...,...,...,...,...,...,...
345,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL92,2018,15.2,NaN
346,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL92,2019,17.0,NaN
347,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL92,2020,17.3,u
348,ESTAT:HTEC_EMP_REG2(1.0),27/04/23 11:00:00,A,HTC,THS_PER,T,PL92,2021,22.0,b


In [110]:
# Dropping some columns
empl_in_tech = empl_in_tech.drop(columns=['DATAFLOW', 'LAST UPDATE', 'freq', 'nace_r2', 'unit', 'OBS_FLAG', 'sex'])
empl_in_tech

,geo,TIME_PERIOD,OBS_VALUE
0,PL2,2008,67.9
1,PL2,2009,71.7
2,PL2,2010,67.4
3,PL2,2011,76.0
4,PL2,2012,86.7
...,...,...,...
345,PL92,2018,15.2
346,PL92,2019,17.0
347,PL92,2020,17.3
348,PL92,2021,22.0


In [111]:
empl_in_tech

,geo,TIME_PERIOD,OBS_VALUE
0,PL2,2008,67.9
1,PL2,2009,71.7
2,PL2,2010,67.4
3,PL2,2011,76.0
4,PL2,2012,86.7
...,...,...,...
345,PL92,2018,15.2
346,PL92,2019,17.0
347,PL92,2020,17.3
348,PL92,2021,22.0


In [107]:
mapping_data

,Unnamed: 0,NUTS1,NUTS1 CODE,NUTS2,NUTS2 CODE,NUTS3,NUTS3 CODE
0,0,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION WROCŁAWSKI,PL619
1,1,MAKROREGION WSCHODNI,PL8,REGION PODLASKIE,PL84,PODREGION SUWALSKI,PL843
2,2,MAKROREGION PÓŁNOCNY,PL6,REGION WARMIŃSKO-MAZURSKIE,PL62,PODREGION OLSZTYŃSKI,PL622
3,3,MAKROREGION POŁUDNIOWY,PL2,REGION ŚLĄSKIE,PL22,PODREGION SOSNOWIECKI,PL22B
4,4,MAKROREGION CENTRALNY,PL7,REGION ŁÓDZKIE,PL71,PODREGION PIOTRKOWSKI,PL713
...,...,...,...,...,...,...,...
67,67,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION WIELKOPOLSKIE,PL41,PODREGION MIASTO POZNAŃ,PL415
68,68,MAKROREGION PÓŁNOCNY,PL6,REGION KUJAWSKO-POMORSKIE,PL61,PODREGION ŚWIECKI,PL618
69,69,MAKROREGION PÓŁNOCNO-ZACHODNI,PL4,REGION ZACHODNIOPOMORSKIE,PL42,PODREGION MIASTO SZCZECIN,PL424
70,70,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION WARSZAWSKI STOŁECZNY,PL91,PODREGION MIASTO WARSZAWA,PL911


In [113]:
column1 = empl_in_tech['geo']
column2 = mapping_data['NUTS2 CODE']

# Finding common values
common_values = set(column1).intersection(set(column2))
common_values

{'PL21',
 'PL22',
 'PL41',
 'PL42',
 'PL43',
 'PL51',
 'PL52',
 'PL61',
 'PL62',
 'PL63',
 'PL71',
 'PL72',
 'PL81',
 'PL82',
 'PL84',
 'PL91',
 'PL92'}

In [114]:
empl_in_tech.rename(columns={'geo': 'NUTS2 CODE'}, inplace=True)

# Performing the merge
merged_dataset = pd.merge(empl_in_tech, merged_df, on='NUTS2 CODE', how='inner')

# Displaying the merged dataset
merged_dataset

,NUTS2 CODE,TIME_PERIOD,OBS_VALUE,Unnamed: 0,NUTS1,NUTS1 CODE,NUTS2,NUTS3,NUTS3 CODE,Kod
0,PL21,2008,35.9,13.0,MAKROREGION POŁUDNIOWY,PL2,REGION MAŁOPOLSKIE,PODREGION KRAKOWSKI,PL214,11212000000
1,PL21,2008,35.9,17.0,MAKROREGION POŁUDNIOWY,PL2,REGION MAŁOPOLSKIE,PODREGION TARNOWSKI,PL217,11212400000
2,PL21,2008,35.9,27.0,MAKROREGION POŁUDNIOWY,PL2,REGION MAŁOPOLSKIE,PODREGION OŚWIĘCIMSKI,PL21A,11212300000
3,PL21,2008,35.9,41.0,MAKROREGION POŁUDNIOWY,PL2,REGION MAŁOPOLSKIE,PODREGION NOWOSĄDECKI,PL218,11212200000
4,PL21,2008,35.9,56.0,MAKROREGION POŁUDNIOWY,PL2,REGION MAŁOPOLSKIE,PODREGION MIASTO KRAKÓW,PL213,11212100000
...,...,...,...,...,...,...,...,...,...,...
1055,PL92,2022,20.0,28.0,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION MAZOWIECKI REGIONALNY,PODREGION CIECHANOWSKI,PL922,71422500000
1056,PL92,2022,20.0,33.0,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION MAZOWIECKI REGIONALNY,PODREGION SIEDLECKI,PL925,71427100000
1057,PL92,2022,20.0,43.0,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION MAZOWIECKI REGIONALNY,PODREGION PŁOCKI,PL923,71427000000
1058,PL92,2022,20.0,46.0,MAKROREGION WOJEWÓDZTWO MAZOWIECKIE,PL9,REGION MAZOWIECKI REGIONALNY,PODREGION ŻYRARDOWSKI,PL926,71427300000


In [115]:
merged_dataset = merged_dataset[['TIME_PERIOD', 'OBS_VALUE', 'Kod']]
merged_dataset.rename(columns={'OBS_VALUE': 'Employment in technology [THS_PER]'}, inplace=True)
merged_dataset

/var/folders/vn/7p3t4_3n02gcv591cgws1gw80000gn/T/ipykernel_18152/222513926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dataset.rename(columns={'OBS_VALUE': 'Employment in technology [THS_PER]'}, inplace=True)


,TIME_PERIOD,Employment in technology [THS_PER],Kod
0,2008,35.9,11212000000
1,2008,35.9,11212400000
2,2008,35.9,11212300000
3,2008,35.9,11212200000
4,2008,35.9,11212100000
...,...,...,...
1055,2022,20.0,71422500000
1056,2022,20.0,71427100000
1057,2022,20.0,71427000000
1058,2022,20.0,71427300000


### Merging  Factors NUTS 2 

In [116]:
factors_1 = melted_rynek.merge(melted_tech_uni, on =['TIME_PERIOD', 'Nazwa', 'Kod'], how ='left')
factors_1

,Kod,Nazwa,TIME_PERIOD,Sites of Worky [tys. places],Uni_tech graduates[person.]
0,0,POLSKA,2022,"534,8",NaN
1,200000,DOLNOŚLĄSKIE,2022,"40,0",NaN
2,400000,KUJAWSKO-POMORSKIE,2022,"21,3",NaN
3,600000,LUBELSKIE,2022,"19,5",NaN
4,800000,LUBUSKIE,2022,"11,9",NaN
...,...,...,...,...,...
250,2400000,ŚLĄSKIE,2008,"68,9",10790.0
251,2600000,ŚWIĘTOKRZYSKIE,2008,"7,2",1225.0
252,2800000,WARMIŃSKO-MAZURSKIE,2008,"15,1",NaN
253,3000000,WIELKOPOLSKIE,2008,"51,7",3895.0


In [117]:
factors_1.isnull().sum()

Kod                              0
Nazwa                            0
TIME_PERIOD                      0
Sites of Worky [tys. places]     0
Uni_tech graduates[person.]     79
dtype: int64

In [118]:
factors_2 = factors_1.merge(melted_tech_wom, on =['TIME_PERIOD', 'Nazwa', 'Kod'], how ='left')
factors_2

,Kod,Nazwa,TIME_PERIOD,Sites of Worky [tys. places],Uni_tech graduates[person.],Women - Uni_tech graduates [person.]
0,0,POLSKA,2022,"534,8",NaN,NaN
1,200000,DOLNOŚLĄSKIE,2022,"40,0",NaN,NaN
2,400000,KUJAWSKO-POMORSKIE,2022,"21,3",NaN,NaN
3,600000,LUBELSKIE,2022,"19,5",NaN,NaN
4,800000,LUBUSKIE,2022,"11,9",NaN,NaN
...,...,...,...,...,...,...
250,2400000,ŚLĄSKIE,2008,"68,9",10790.0,4482.0
251,2600000,ŚWIĘTOKRZYSKIE,2008,"7,2",1225.0,363.0
252,2800000,WARMIŃSKO-MAZURSKIE,2008,"15,1",NaN,NaN
253,3000000,WIELKOPOLSKIE,2008,"51,7",3895.0,1100.0


In [119]:
factors = factors_2.to_excel('all_factors_nuts2.xlsx')